### Install package

In [ ]:
!pip install pytorch-adapt[lightning]

### Import packages

In [ ]:
import pytorch_lightning as pl
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.frameworks.lightning import Lightning
from pytorch_adapt.frameworks.utils import filter_datasets
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.validators import IMValidator

### Create datasets and dataloaders

In [ ]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".", download=True)
dc = DataloaderCreator(batch_size=32, num_workers=2)

### Create models, optimizers, hook, and validator

In [ ]:
G = mnistG(pretrained=True)
C = mnistC(pretrained=True)
D = Discriminator(in_size=1200, h=256)
models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.0001}))

adapter = DANN(models=models, optimizers=optimizers)
validator = IMValidator()
dataloaders = dc(**filter_datasets(datasets, validator))
train_loader = dataloaders.pop("train")

L_adapter = Lightning(adapter, validator=validator)
trainer = pl.Trainer(gpus=1, max_epochs=2)

### Train and evaluate

In [ ]:
trainer.fit(L_adapter, train_loader, *dataloaders.values())